In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import warnings
import keras
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [3]:
train1 = pd.read_csv('train.csv')
test1 = pd.read_csv('test.csv')
train1.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [4]:
def cleandata(df):
    df.Age=df.Age.replace(0,np.NaN)
    df.Embarked=df.Embarked.replace(np.NaN,'S')
    df.fillna(df.median(), inplace=True)
    Gender=pd.get_dummies(df.Sex,drop_first=True)
    Pclass_F=pd.get_dummies(df.Pclass,'Pclass',drop_first=True)
    Embarked_F=pd.get_dummies(df.Embarked,drop_first=True)
    df=pd.concat([df,Gender,Pclass_F,Embarked_F],1)
    df=df.drop(['Cabin','Ticket','Name','Sex','Embarked','Pclass'],1)
    return(df)

In [5]:
train2=cleandata(train1)
test2=cleandata(test1)
train2.head(2)

,PassengerId,Survived,Age,SibSp,Parch,Fare,male,Pclass_2,Pclass_3,Q,S
0,1,0,22.0,1,0,7.2500,1,0,1,0,1
1,2,1,38.0,1,0,71.2833,0,0,0,0,0


In [6]:
from sklearn.preprocessing import StandardScaler
featurelist=['Age','Fare']
ssc=StandardScaler()
train2[featurelist]=ssc.fit_transform(train2[featurelist])
test2[featurelist]=ssc.transform(test2[featurelist])
train2.head(2)

,PassengerId,Survived,Age,SibSp,Parch,Fare,male,Pclass_2,Pclass_3,Q,S
0,1,0,-0.565736,1,0,-0.502445,1,0,1,0,1
1,2,1,0.663861,1,0,0.786845,0,0,0,0,0


In [7]:
featurelist=['Age','SibSp','Fare','Parch','male','Pclass_2','Pclass_3','Q','S']
X=train2[featurelist]
Xtest=test2[featurelist]
y=train2.Survived

In [28]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

model=Sequential()
model.add(Dense(units=5,kernel_initializer='uniform',activation='relu',input_dim=9))
model.add(Dense(units=5,kernel_initializer='uniform',activation='relu'))
model.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))
model.summary

<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x000001860BAC8978>>

In [31]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
history=model.fit(X,y,batch_size=10,nb_epoch=100)

Epoch 1/100
891/891 [==============================] - 1s 868us/step - loss: 0.6896 - acc: 0.5982
Epoch 2/100
891/891 [==============================] - 0s 319us/step - loss: 0.6772 - acc: 0.6229
Epoch 3/100
891/891 [==============================] - 0s 293us/step - loss: 0.6579 - acc: 0.6465
Epoch 4/100
891/891 [==============================] - 0s 300us/step - loss: 0.6321 - acc: 0.6981
Epoch 5/100
891/891 [==============================] - 0s 316us/step - loss: 0.6004 - acc: 0.7351
Epoch 6/100
891/891 [==============================] - 0s 308us/step - loss: 0.5690 - acc: 0.7621
Epoch 7/100
891/891 [==============================] - 0s 327us/step - loss: 0.5385 - acc: 0.7744
Epoch 8/100
891/891 [==============================] - 0s 333us/step - loss: 0.5135 - acc: 0.7845
Epoch 9/100
891/891 [==============================] - 0s 324us/step - loss: 0.4946 - acc: 0.7980
Epoch 10/100
891/891 [==============================] - 0s 330us/step - loss: 0.4808 - acc: 0.7969
Epoch 11/100
891/89

891/891 [==============================] - 0s 230us/step - loss: 0.4033 - acc: 0.8249
Epoch 84/100
891/891 [==============================] - 0s 207us/step - loss: 0.4043 - acc: 0.8260
Epoch 85/100
891/891 [==============================] - 0s 230us/step - loss: 0.4020 - acc: 0.8227
Epoch 86/100
891/891 [==============================] - 0s 214us/step - loss: 0.4034 - acc: 0.8260
Epoch 87/100
891/891 [==============================] - 0s 214us/step - loss: 0.4032 - acc: 0.8260
Epoch 88/100
891/891 [==============================] - 0s 230us/step - loss: 0.4029 - acc: 0.8215
Epoch 89/100
891/891 [==============================] - 0s 266us/step - loss: 0.4028 - acc: 0.8272
Epoch 90/100
891/891 [==============================] - 0s 253us/step - loss: 0.4026 - acc: 0.8249
Epoch 91/100
891/891 [==============================] - 0s 237us/step - loss: 0.4030 - acc: 0.8260
Epoch 92/100
891/891 [==============================] - 0s 282us/step - loss: 0.4021 - acc: 0.8215
Epoch 93/100
891/891 [=

In [22]:
yhat=model.predict(Xtest)
Survived=(yhat>0.5)

In [23]:
test1['Survived']=Survived
testpred=test1.drop(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],1)
testpred.to_csv('keraspred.csv',sep=',')

In [32]:
#Kfold Cross Val

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
def build_model():
    model=Sequential()
    model.add(Dense(units=5,kernel_initializer='uniform',activation='relu',input_dim=9))
    model.add(Dense(units=5,kernel_initializer='uniform',activation='relu'))
    model.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

new_model=KerasClassifier(build_fn=build_model,epochs=100,batch_size=10)
accuracies=cross_val_score(estimator=new_model,X=X,y=y,cv=10,n_jobs=-1,verbose=1)
print(f'{accuracies},\nmean={accuracies.mean()},\nvar={accuracies.std()}')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:  1.2min remaining:   48.1s


[0.76666666 0.82022471 0.79775281 0.80898876 0.80898876 0.80898876
 0.78651685 0.76404495 0.85393257 0.84269663],
mean=0.8058801453360607,
var=0.027619047585337893


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.5min finished


In [38]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
def build_model():
    model=Sequential()
    model.add(Dense(units=5,kernel_initializer='uniform',activation='relu',input_dim=9))
    model.add(Dense(units=5,kernel_initializer='uniform',activation='relu'))
    model.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model
new_model=KerasClassifier(build_fn=build_model)
param={'batch_size':[10,25,32,40,50],'nb_epoch':[100,200,300,500]}
gd=GridSearchCV(estimator=new_model,param_grid=param,scoring='accuracy',cv=10,verbose=1)
gd=gd.fit(X,y)

print(gd.best_params_)
print(gd.best_score_)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/1
801/801 [==============================] - 1s 980us/step - loss: 0.6893 - acc: 0.6167
Epoch 1/1
802/802 [==============================] - 1s 966us/step - loss: 0.6905 - acc: 0.5910
Epoch 1/1
802/802 [==============================] - 1s 938us/step - loss: 0.6894 - acc: 0.6160
Epoch 1/1
802/802 [==============================] - 1s 1ms/step - loss: 0.6873 - acc: 0.6247
Epoch 1/1
802/802 [==============================] - 1s 1ms/step - loss: 0.6899 - acc: 0.6160
Epoch 1/1
802/802 [==============================] - 1s 1ms/step - loss: 0.6888 - acc: 0.6110
Epoch 1/1
802/802 [==============================] - 1s 1ms/step - loss: 0.6881 - acc: 0.6209
Epoch 1/1
802/802 [==============================] - 1s 1ms/step - loss: 0.6892 - acc: 0.6147
Epoch 1/1
802/802 [==============================] - 1s 1ms/step - loss: 0.6900 - acc: 0.6085
Epoch 1/1
802/802 [==============================] - 1s 1ms/step - loss: 0.6898 - acc: 0.6147
Epoch 1/1
801/801 [==============================] - 1

KeyboardInterrupt: 